In [1]:
import pandas as pd

df = pd.read_parquet("green_tripdata_2025-11.parquet")

df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])





8007

In [2]:
# question 3
result = df[
    (df["lpep_pickup_datetime"] >= "2025-11-01") &
    (df["lpep_pickup_datetime"] < "2025-12-01") &
    (df["trip_distance"] <= 1)
]

len(result)

8007

In [3]:
# question 4
df_valid = df[df["trip_distance"] < 100]
df_valid["pickup_day"] = df_valid["lpep_pickup_datetime"].dt.date

longest_trip = df_valid.loc[df_valid["trip_distance"].idxmax()]
print(longest_trip["pickup_day"])

2025-11-14


In [4]:
# question 5
zones = pd.read_csv("taxi_zone_lookup.csv")

df_18 = df[
    (df["lpep_pickup_datetime"] >= "2025-11-18") &
    (df["lpep_pickup_datetime"] < "2025-11-19")
]

total_by_zone = df_18.groupby("PULocationID")["total_amount"].sum().reset_index()

total_by_zone = total_by_zone.merge(
    zones,
    left_on="PULocationID",
    right_on="LocationID",
    how="left"
)

max_zone = total_by_zone.loc[total_by_zone["total_amount"].idxmax()]
print(max_zone["Zone"])

East Harlem North


In [5]:
# question 6
# Strip any whitespace from Zone column
zones["Zone"] = zones["Zone"].str.strip()

# Filter November 2025
df_nov = df[
    (df["lpep_pickup_datetime"] >= "2025-11-01") &
    (df["lpep_pickup_datetime"] < "2025-12-01")
]

# Merge pickup zone
df_nov = df_nov.merge(
    zones[["LocationID", "Zone"]],
    left_on="PULocationID",
    right_on="LocationID",
    how="left"
)
df_nov.rename(columns={"Zone": "pickup_zone"}, inplace=True)

# Filter East Harlem North pickups
df_pickup_ehn = df_nov[df_nov["pickup_zone"] == "East Harlem North"]
print("Number of trips:", len(df_pickup_ehn))

if len(df_pickup_ehn) > 0:
    largest_tip_trip = df_pickup_ehn.loc[df_pickup_ehn["tip_amount"].idxmax()]
    dropoff_zone_id = largest_tip_trip["DOLocationID"]
    dropoff_zone_name = zones.loc[zones["LocationID"] == dropoff_zone_id, "Zone"].values[0]
    print("Dropoff zone with largest tip:", dropoff_zone_name)
else:
    print("No trips found in East Harlem North in November 2025")


Number of trips: 12041
Dropoff zone with largest tip: Yorkville West
